### Notes

Note, this requires using PyTorch v0.3.1.  Somewhere between 0.3.1 and 0.4.0 parts of the backend were significantly rewritten, preventing us from performing the following hacks. (Likely has to do with them fusing Variable and Tensor).  That may change once their new API stabilizes.

The nice thing about how this is working is that it should be general enough to work for compute, tree, and federated modes of Grid, depending on how the `receive` function works under the hood.

In [1]:
import torch
import inspect
from torch.autograd import Variable
import random
import re
from functools import wraps, partial, partialmethod
from types import *
import imp
# from contextlib import contextmanager

print(torch.__version__)

0.3.0.post4


In [2]:
from grid.clients.torch import TorchClient
client = TorchClient(verbose = False)

/Users/morgangiraud/.virtualenvs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-macosx-10.13-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.



UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: QmXhfzHqbtMNTzYHGC31Fdd3tTuMzwbGYgn671GAxeoRer

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/QmXkWUybbTnfvFH8SUcrug6RGTLYTB23gSockKLxueR1vQ...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/Qme8SQLibzaAPQSS4GRFQCqAXqVPVknZeDLPqeePYYka8d...FAIL!!!
	WORKER: /p2p-circuit/ipfs/Qmaosc64H6Y29VFCFYJzJXCX9AuRp7RCsekLmajHNVEARD...FAIL!!!
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...SUCCESS!!!

UPDATE: Searching for IPFS nodes - 14 found overall - 1 are OpenMined workers          

SUCCESS: Found 1 OpenMined nodes!!!



In [3]:
service_self = client.services['torch_service']

In [4]:
tensor_types = [torch.FloatTensor,
                torch.DoubleTensor,
                torch.HalfTensor,
                torch.ByteTensor,
                torch.CharTensor,
                torch.ShortTensor,
                torch.IntTensor,
                torch.LongTensor]

In [5]:
def assign_workers_function(worker_ids):
    def decorate(func):
        @wraps(func)
        def send_to_workers(*args, **kwargs):
            part = func(*args, **kwargs)
            command = compile_command(part)
            for worker in worker_ids:
                print("Placeholder print for sending command to worker {}".format(worker))
                args, kwargs = send_command(command)
            receive_commands(worker_ids)  ## Probably needs to happen async
            return args, kwargs
        return send_to_workers
    return decorate

In [6]:
def assign_workers_method(worker_ids):
    def decorate(method):
        @wraps(method)
        def send_to_workers(self, *args, **kwargs):
            part = method(self, *args, **kwargs)
            if self.is_pointer_to_remote:
                command = compile_command(part)
                for worker in worker_ids:
                    print("Placeholder print for sending command to worker {}".format(worker))
                    args, kwargs = send_command(command)
                receive_commands(worker_ids)  ## Probably needs to happen async
                return args, kwargs
            else:
                result = part.func(self, *args, **kwargs)
                my_service = self.worker.services['torch_service']
                if type(result) in tensor_types:
                    my_service.register_object(result, False)
                elif type(result) == torch.autograd.variable.Variable:
                    import ipdb; ipdb.set_trace()
                    my_service.register_object(result.data, False)
                    if result.requires_grad is True and result.volatile is False:
                        def var_hook(grad):
                            import ipdb; ipdb.set_trace()
                            print("hooking on:", self.id)
                            if not hasattr(grad, 'owner'):
                                my_service.register_object(grad, False)
                            return grad
                        result.register_hook(var_hook)
                return result
        return send_to_workers
    return decorate

In [7]:
def pass_func_args(func):
    @wraps(func)
    def pass_args(*args, **kwargs):
        return partial(func, *args, **kwargs)
    return pass_args

def pass_method_args(method):
    @wraps(method)
    def pass_args(*args, **kwargs):
        return partialmethod(method, *args, **kwargs)
    return pass_args

In [8]:
def send_command(command):
    print(command['command'])
    print([type(arg) for arg in command['args']])
    print([type(pair) for pair in command['kwargs']])
    print('===========')
    print()
    return command['args'], command['kwargs']

In [9]:
def receive_commands(worker_ids):
    print('Placeholder print for receiving commands from workers in the following list')
    print(worker_ids)

In [10]:
def compile_command(partial_func):
    func = partial_func.func
    args = partial_func.args
    kwargs = partial_func.keywords
    command = {}
    command['command'] = func.__name__
    command['command_type'] = type(func)
    command['args'] = args
    command['kwargs'] = kwargs
    command['arg_types'] = [type(x) for x in args]
    command['kwarg_types'] = [type(kwargs[x]) for x in kwargs]
    return command

In [11]:
x = torch.FloatTensor([1,1])

In [12]:
%%time
for x in range(100000):
    y = torch.FloatTensor([[2,2],[2,2]])
    z = torch.FloatTensor([[1,1],[1,1]])
    res = y.add(z)
    

CPU times: user 2.18 s, sys: 86.7 ms, total: 2.26 s
Wall time: 2.29 s


In [13]:
%%time

for attr in dir(torch):
    if attr == 'typename':
        continue
    if type(torch.__getattribute__(attr)) in [FunctionType, BuiltinFunctionType]:
        torch.__setattr__(attr, assign_workers_function(['A1','B1', 'B2'])(pass_func_args(torch.__getattribute__(attr))))

exclude = ['ndimension', 'nelement', 'size','numel', 'ser', 'de']
skipped = []
overloaded = []
for attr in dir(torch.FloatTensor):
    lit = getattr(torch.FloatTensor, attr)
    is_desc = inspect.ismethoddescriptor(lit)
    is_func = type(lit)==FunctionType
    is_mappingproxy = attr == '__dict__'
    try:
        is_service_func = 'TorchService' in lit.__qualname__
    except:
        is_service_func = False
    is_base = attr in dir(object)
    is_old = re.match('old*', attr) is not None
    if attr in exclude:
        skipped.append(attr)
        continue
    if (is_desc or (is_func and not is_service_func)) and not is_base and not is_old:
        overloaded.append(attr)

        setattr(torch.FloatTensor, 'old_{}'.format(attr), lit)
        setattr(torch.FloatTensor, attr, assign_workers_method(['A1','B1', 'B2'])(pass_method_args(lit)))
    else:
        skipped.append(attr)

print("Attribute skipped: %s\n" % ", ".join(skipped))
print("Attribute overloaded: %s\n" % ", ".join(overloaded))

Attribute skipped: __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, _cdata, _new_with_metadata_file, _torch, data, de, get, is_cuda, is_sparse, ndimension, nelement, numel, old__repr__, process_command, send, ser, shape, size, storage_type

Attribute overloaded: __add__, __and__, __array__, __array_wrap__, __bool__, __deepcopy__, __delitem__, __div__, __float__, __getitem__, __getstate__, __iadd__, __iand__, __idiv__, __ilshift__, __imul__, __int__, __invert__, __ior__, __ipow__, __irshift__, __isub__, __iter__, __itruediv__, __ixor__, __len__, __long__, __lshift__, __matmul__, __mod__, __mul__, __neg__, __nonzero__, __or__, __pow__, __radd__, __rdiv__, __rmul__, __rpow__, __rshift__, __rsub__, __rtruediv__, __setitem__, __setstate__, __sub__, __t

In [14]:
%%time
for x in range(100000):
    y = torch.FloatTensor([[2,2],[2,2]])
    z = torch.FloatTensor([[1,1],[1,1]])
    res = y.add(z)

CPU times: user 2.94 s, sys: 113 ms, total: 3.06 s
Wall time: 3.06 s


In [15]:
x = y.add(z)
print(x.is_pointer_to_remote)
print(x.id)

False
961403038


In [16]:
x


 3  3
 3  3
[torch.FloatTensor of size 2x2]

In [17]:
x.fill_(0)


 0  0
 0  0
[torch.FloatTensor of size 2x2]

In [18]:
print(x)


 0  0
 0  0
[torch.FloatTensor of size 2x2]



In [19]:
x.is_pointer_to_remote = True
x.owner = 'other_guy'

In [20]:
x.normal_()

Placeholder print for sending command to worker A1
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
normal_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],), {})

In [21]:
x.uniform_()

Placeholder print for sending command to worker A1
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
uniform_
[<class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],), {})

In [22]:
torch.add(x, x)

Placeholder print for sending command to worker A1
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B1
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for sending command to worker B2
add
[<class 'torch.FloatTensor'>, <class 'torch.FloatTensor'>]
[]

Placeholder print for receiving commands from workers in the following list
['A1', 'B1', 'B2']


(([ torch.FloatTensor - Location:other_guy ],
  [ torch.FloatTensor - Location:other_guy ]),
 {})

### Variable

In [23]:
def new___init__(self,tensor,owner=service_self, *args, **kwargs):
    super(Variable, self).__init__(*args, **kwargs)
    import ipdb; ipdb.set_trace()
    if not hasattr(self.data, 'owner'):
        owner.register_object(self.data,False)
    if self.grad is not None and not hasattr(self.grad, 'owner'):
        owner.register_object(self.grad,False)
    owner.register_object(self,False)
Variable.__init__ = new___init__

def new___setattr__(self, *args, **kwargs):
    print('mutation!', args)
    if args[0] == 'grad':
        import ipdb; ipdb.set_trace()
    return self.old___setattr__(*args, **kwargs)
Variable.old___setattr__ = Variable.__setattr__
Variable.__setattr__ = new___setattr__

exclude = ['ndimension', 'nelement', 'size','numel', 'ser', 'de']
overloaded = []
skipped = []
for attr in dir(Variable):
    lit = getattr(Variable, attr)
    is_desc = inspect.ismethoddescriptor(lit)
    is_func = type(lit)==FunctionType
    is_mappingproxy = attr == '__dict__'
    try:
        is_service_func = 'TorchService' in lit.__qualname__
    except:
        is_service_func = False
    is_base = attr in dir(object)
    is_old = re.match('old*', attr) is not None
    if attr in exclude:
        skipped.append(attr)
        continue
    if (is_desc or (is_func and not is_service_func)) and not is_base and not is_old:
        overloaded.append(attr)
        setattr(Variable, 'old_{}'.format(attr), lit)
        setattr(Variable, attr, assign_workers_method(['A1','B1', 'B2'])(pass_method_args(lit)))
    else:
        skipped.append(attr)

print("Attribute skipped: %s\n" % ", ".join(skipped))
print("Attribute overloaded: %s\n" % ", ".join(overloaded))

Attribute skipped: __class__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__, __subclasshook__, __weakref__, _backward_hooks, _execution_engine, _fallthrough_methods, _grad, _grad_fn, _torch, _version, cat, data, grad, grad_fn, is_leaf, numel, old___setattr__, ones_like, output_nr, requires_grad, volatile, zeros_like

Attribute overloaded: __add__, __and__, __bool__, __deepcopy__, __div__, __float__, __getattr__, __getitem__, __iadd__, __iand__, __idiv__, __ilshift__, __imul__, __int__, __ior__, __ipow__, __irshift__, __isub__, __iter__, __ixor__, __len__, __long__, __lshift__, __matmul__, __mod__, __mul__, __neg__, __nonzero__, __or__, __pow__, __radd__, __rdiv__, __rmul__, __rpow__, __rshift__, __rsub__, __rtruediv__, __setitem__, __setstate__, __sub__, __truediv__, __xor__, _advanced_

In [24]:
x = torch.FloatTensor([1,1])
print("x.owner: %s" % x.owner)
v = Variable(x, requires_grad=True)
print("v.data.owner: %s" % v.data.owner)

x.owner: QmXhfzHqbtMNTzYHGC31Fdd3tTuMzwbGYgn671GAxeoRer
> <ipython-input-23-21db014a1001>(4)new___init__()
      3     import ipdb; ipdb.set_trace()
----> 4     if not hasattr(self.data, 'owner'):
      5         owner.register_object(self.data,False)

ipdb> self
Variable containing:
 1
 1
[torch.FloatTensor of size 2]

ipdb> self.data

 1
 1
[torch.FloatTensor of size 2]

ipdb> self.data.owner
'QmXhfzHqbtMNTzYHGC31Fdd3tTuMzwbGYgn671GAxeoRer'
ipdb> c
mutation! ('id', 4827823782)
mutation! ('owner', 'QmXhfzHqbtMNTzYHGC31Fdd3tTuMzwbGYgn671GAxeoRer')
mutation! ('worker', <grid.clients.torch.TorchClient object at 0x10e644668>)
mutation! ('is_pointer_to_remote', False)
v.data.owner: QmXhfzHqbtMNTzYHGC31Fdd3tTuMzwbGYgn671GAxeoRer


In [25]:
v.is_pointer_to_remote

False

In [26]:
type(v)

torch.autograd.variable.Variable

In [31]:
z = v * 2
print(
    z.is_pointer_to_remote, 
    z.data.is_pointer_to_remote, 
    z.data.owner
)

> <ipython-input-6-1450201fc71f>(20)send_to_workers()
     19                     import ipdb; ipdb.set_trace()
---> 20                     my_service.register_object(result.data, False)
     21                     if result.requires_grad is True and result.volatile is False:

ipdb> c


RecursionError: maximum recursion depth exceeded

In [ ]:
v.volatile

In [ ]:
z

In [29]:
z.backward()

AttributeError: 'torch.FloatTensor' object has no attribute 'backward'

In [ ]:
print(v.grad)

> /Users/morgangiraud/Sites/openmined/Grid/grid/services/torch/torch_service.py(228)__repr__()
    227                 import ipdb; ipdb.set_trace()
--> 228             if(service_self.worker.id == self.owner):
    229                 return self.old__repr__()

ipdb> self
*** AttributeError: 'torch.FloatTensor' object has no attribute 'owner'
ipdb> self.owner
*** AttributeError: 'torch.FloatTensor' object has no attribute 'owner'
ipdb> type(self)
<class 'torch.FloatTensor'>
ipdb> w
  /usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py(193)_run_module_as_main()
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
    194 

  /usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py(85)_run_code()
     84                        __spec__ = mod_spec)
---> 85     exec(code, run_globals)
     86     return run_globals

  /Users/morgangiraud/.virtualenvs/ope